In [3]:
import duckdb 
query = """
	with recursive mvt as materialized(
select
	distinct mod,
	date_eff,
	com_av,
	libelle_av,
	com_ap,
	libelle_ap
from
	read_csv('https://www.insee.fr/fr/statistiques/fichier/7766585/v_mvt_commune_2024.csv')
where
	typecom_av = 'COM'
	and typecom_ap = 'COM'
	and not (mod <> 21
		and com_av = com_ap)
	and mod not in (20, 30)
	),
	start_com as (
select
	*
from
	(
	select
		*,
		max(date_eff) over(partition by com_av) as firstdate
	from
		mvt)
where
	date_eff = firstdate),
	com_cte(com_av,
libelle_av,
date_eff,
com_av2,
com_ap,
libelle_ap,
count_) as (
select
	start_com.com_av as com_av,
	start_com.libelle_av as libelle_av,
	start_com.date_eff as date_eff,
	start_com.com_av as com_av2 ,
	start_com.com_ap as com_ap,
	start_com.libelle_ap as libelle_ap,
	1 as count_
from
	start_com
UNION ALL
select
	com_cte.com_av as com_av,
	com_cte.libelle_av as libelle_av,
	mt.date_eff as date_eff,
	mt.com_av as com_av2 ,
	mt.com_ap as com_ap,
	mt.libelle_ap as libelle_ap,
	com_cte.count_ + 1 as count_
from
	com_cte
join mvt as mt on
	com_cte.com_ap = mt.com_av
	and com_cte.date_eff <= mt.date_eff
where
	not (mt.mod = 21
		and com_av2 <> mt.com_ap)
	and count_ < 10
	),
	final_result as (
select
	distinct com_av,
	libelle_av,
	com_ap,
	libelle_ap
from
	(
	select
		*,
		max(count_) over(partition by com_av) as maxcount
	from
		com_cte)
where
	count_ = maxcount)
	select
	*
from
	final_result
order by com_av asc
"""
duckdb.sql(query).write_csv("passage_communes_1943.csv")
duckdb.sql(query).show()

┌─────────┬────────────────────────────────────┬─────────┬────────────────────────────────┐
│ com_av  │             libelle_av             │ com_ap  │           libelle_ap           │
│ varchar │              varchar               │ varchar │            varchar             │
├─────────┼────────────────────────────────────┼─────────┼────────────────────────────────┤
│ 01003   │ Amareins                           │ 01165   │ Amareins-Francheleins-Cesseins │
│ 01018   │ Arlod                              │ 01033   │ Bellegarde-sur-Valserine       │
│ 01020   │ Arnans                             │ 01125   │ Corveissiat                    │
│ 01039   │ Béon                               │ 01138   │ Culoz-Béon                     │
│ 01048   │ Bohas                              │ 01245   │ Bohas-Meyriat-Rignat           │
│ 01055   │ Bouvent                            │ 01283   │ Oyonnax                        │
│ 01059   │ Brénaz                             │ 01453   │ Arvière-en-Valromey  